<a href="https://colab.research.google.com/github/aaayushisharmaaa/Pokemon-Battle-Predictor/blob/main/Pokemon_Battle_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pokemon Battle Winner Predictor

## Datasets
combats.csv (train data) = https://raw.githubusercontent.com/saurabhcharde/Pokemon_Battle_Winner_Prediction_using_ML/master/datasets/combats.csv

pokemon_data.csv (features) = https://raw.githubusercontent.com/saurabhcharde/Pokemon_Battle_Winner_Prediction_using_ML/master/datasets/pokemons_data.csv

tests.csv (testing data) = https://raw.githubusercontent.com/saurabhcharde/Pokemon_Battle_Winner_Prediction_using_ML/master/datasets/tests.csv

## Step 1: import data

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
pokemon=pd.read_csv('https://raw.githubusercontent.com/saurabhcharde/Pokemon_Battle_Winner_Prediction_using_ML/master/datasets/pokemons_data.csv',index_col=0)
combats=pd.read_csv('https://raw.githubusercontent.com/saurabhcharde/Pokemon_Battle_Winner_Prediction_using_ML/master/datasets/combats.csv')

In [ ]:
pokemon.head()

,Name,Hit Points,Attack,Defense,Sp. Atk,Sp. Def,Speed,Legendary
#,,,,,,,,
1,Bulbasaur,45,49,49,65,65,45,0
2,Ivysaur,60,62,63,80,80,60,0
3,Venusaur,80,82,83,100,100,80,0
4,Mega Venusaur,80,100,123,122,120,80,0
5,Charmander,39,52,43,60,50,65,0


## Step 2: Data preprocessing

In [ ]:
cols = ["First_pokemon","Second_pokemon","Winner"]
new_combat_data=combats[cols].replace(pokemon.Name)
new_combat_data.head()

,First_pokemon,Second_pokemon,Winner
0,Larvitar,Nuzleaf,Nuzleaf
1,Virizion,Terrakion,Terrakion
2,Togetic,Beheeyem,Beheeyem
3,Slugma,Druddigon,Druddigon
4,Omastar,Shuckle,Omastar


In [ ]:
combats.Winner[combats.Winner == combats.First_pokemon] = 0
combats.Winner[combats.Winner == combats.Second_pokemon] = 1

## Step 3: Normalising data and adding features

In [ ]:
def normalization(data_df):
    stats=["Hit Points","Attack","Defense","Sp. Atk","Sp. Def","Speed","Legendary"]
    stats_df=pokemon[stats].T.to_dict("list")
    one=data_df.First_pokemon.map(stats_df)
    print(one, len(one))
    two=data_df.Second_pokemon.map(stats_df)
    print(two, len(two))
    temp_list=[]
    for i in range(len(one)):
        temp_list.append(np.array(one[i])-np.array(two[i]))
        print(temp_list)
    new_test = pd.DataFrame(temp_list, columns=stats)
    for c in stats:
        description=new_test[c].describe()
        print(description)
        new_test[c]=(new_test[c]-description['min'])/(description['max']-description['min'])
    return new_test

In [ ]:
data=normalization(combats)
data = pd.concat([data,combats.Winner], axis=1)
data.head()

,Hit Points,Attack,Defense,Sp. Atk,Sp. Def,Speed,Legendary,Winner
0,0.455556,0.468571,0.517241,0.450425,0.527578,0.450746,0.5,1
1,0.500000,0.374286,0.452874,0.543909,0.597122,0.507463,0.5,1
2,0.455556,0.385714,0.517241,0.365439,0.527578,0.507463,0.5,1
3,0.417778,0.257143,0.379310,0.521246,0.383693,0.423881,0.5,1
4,0.611111,0.628571,0.252874,0.790368,0.119904,0.656716,0.5,0


## Step 4: Train-Test Split

In [ ]:
x_label=data.drop("Winner",axis=1)
y_label=data["Winner"]

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_label, y_label, test_size=0.25, random_state=42)

## Step 5: Training the model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
clf = RandomForestClassifier(n_estimators=100)
model = clf.fit(x_train, y_train)
pred = model.predict(x_test)
print('Accuracy of model: ', accuracy_score(pred, y_test)*100)

Accuracy of model:  94.992


## Step 6: Testing the model

In [ ]:
test_data=pd.read_csv('https://raw.githubusercontent.com/saurabhcharde/Pokemon_Battle_Winner_Prediction_using_ML/master/datasets/tests.csv')


In [ ]:
new_test_data=test_data[["First_pokemon","Second_pokemon"]].replace(pokemon.Name)
new_test_data.head()

,First_pokemon,Second_pokemon
0,Seaking,Lickitung
1,Ferrothorn,Quagsire
2,Thundurus Therian Forme,Hitmonlee
3,Flaaffy,Maractus
4,Spearow,Alomomola


In [ ]:
final_data=normalization(test_data)
pred=model.predict(final_data)
test_data["Winner"]=[test_data["First_pokemon"][i] if pred[i]==0 else test_data["Second_pokemon"][i] for i in range(len(pred))]

In [ ]:
combats_name = test_data[cols].replace(pokemon.Name)
combats_name[63:64]

,First_pokemon,Second_pokemon,Winner
63,Mr. Mime,Psyduck,Mr. Mime


## Step 7: Making the predictor

In [ ]:
first_pokemon = input("Enter first pokemon: ")
second_pokemon = input("Enter second pokemon: ")
new_test_data = pd.DataFrame({"First_pokemon": [first_pokemon.title()], "Second_pokemon": [second_pokemon.title()]})
new_test_data

Enter first pokemon: charmander
Enter second pokemon: bulbasaur


,First_pokemon,Second_pokemon
0,Charmander,Bulbasaur


In [ ]:
first_dex = pokemon[pokemon['Name'] == first_pokemon.title()].index
second_dex = pokemon[pokemon['Name'] == second_pokemon.title()].index
test_data = pd.DataFrame({"First_pokemon": [first_dex.tolist()[0]], "Second_pokemon": [second_dex.tolist()[0]]})
test_data

,First_pokemon,Second_pokemon
0,5,1


In [ ]:
def new_normalization(data_df):
    stats=["Hit Points","Attack","Defense","Sp. Atk","Sp. Def","Speed","Legendary"]
    stats_df=pokemon[stats].T.to_dict("list")
    one=data_df.First_pokemon.map(stats_df)
    print(one, len(one))
    two=data_df.Second_pokemon.map(stats_df)
    print(two, len(two))
    temp_list=[]
    for i in range(len(one)):
        temp_list.append(np.array(one[i])-np.array(two[i]))
        print(temp_list)
    new_test = pd.DataFrame(temp_list, columns=stats)
    return new_test

In [ ]:
final_data=new_normalization(test_data)
print(final_data)
pred=model.predict(final_data)
test_data["Winner"]=[test_data["First_pokemon"][i] if pred[i]==0 else test_data["Second_pokemon"][i] for i in range(len(pred))]
combats_name = test_data[cols].replace(pokemon.Name)
combats_name

0    [39, 52, 43, 60, 50, 65, 0]
Name: First_pokemon, dtype: object 1
0    [45, 49, 49, 65, 65, 45, 0]
Name: Second_pokemon, dtype: object 1
[array([ -6,   3,  -6,  -5, -15,  20,   0])]
   Hit Points  Attack  Defense  Sp. Atk  Sp. Def  Speed  Legendary
0          -6       3       -6       -5      -15     20          0


,First_pokemon,Second_pokemon,Winner
0,Charmander,Bulbasaur,Charmander
